<a href="https://colab.research.google.com/github/Diezelots/GDG/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor


In [13]:
train_df = pd.read_csv('/content/drive/MyDrive/서강대 화공/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/서강대 화공/test.csv')
train_df.head()
test_df.head()
train_df.info()

train_df['is_train'] = 1
test_df['is_train'] = 0
test_df['Survived'] = np.nan

#train/test set 묶어서 결측치, 범주형 등 전처리
combined = pd.concat([train_df, test_df], sort=False)
combined['Title'] = combined['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
combined['Title'] = combined['Title'].replace([
    'Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
    'Rare')
combined['Title'] = combined['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
combined['Title'] = combined['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4})
combined['Title'] = combined['Title'].fillna(0).astype(int)
combined = combined.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
combined['Embarked'].fillna(combined['Embarked'].mode()[0], inplace=True)
combined['Age'].fillna(combined['Age'].median(), inplace=True)
combined['Fare'].fillna(combined['Fare'].median(), inplace=True)
combined['Sex'] = combined['Sex'].map({'male': 0, 'female': 1})
combined['Embarked'] = combined['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1
combined['IsAlone'] = (combined['FamilySize'] == 1).astype(int)

train = combined[combined['is_train'] == 1].drop(['is_train'], axis=1)
test = combined[combined['is_train'] == 0].drop(['is_train', 'Survived'], axis=1)

X_train = train.drop('Survived', axis=1)
y_train = train['Survived'].astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


<ipython-input-13-de07796cb38d>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined['Embarked'].fillna(combined['Embarked'].mode()[0], inplace=True)
<ipython-input-13-de07796cb38d>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(valu

In [14]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(test)

original_test = pd.read_csv('/content/drive/MyDrive/서강대 화공/test.csv')

submission = pd.DataFrame({
    'PassengerId': original_test['PassengerId'],
    'Survived': predictions.astype(int)
})

submission.to_csv('submission.csv', index=False)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 훈련/검증 데이터 분리
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# 모델 훈련
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_tr, y_tr)

# 예측
y_pred = model.predict(X_val)

# 정확도 출력
acc = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {acc:.4f}")

Validation Accuracy: 0.8380


In [4]:
submission = pd.DataFrame({
    'PassengerId': original_test['PassengerId'],
    'Survived': predictions.astype(int)
})
submission.to_csv('submission.csv', index=False)